In [1]:
import json
import re
import pandas as pd
import numpy as np
import catboost as cb
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from nltk import ngrams
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from collections import Counter
from metric import pr_auc_macro
from utils import process_nan, try_cast_to_float, augment
from distances import cosine_distance, cosine_distances, jaccard
import Levenshtein
import jellyfish

from tqdm.auto import tqdm
tqdm().pandas()

0it [00:00, ?it/s]

In [2]:
pairs = pd.read_parquet('../hackathon_files_for_participants_ozon/train_pairs.parquet')
data = pd.read_parquet('../hackathon_files_for_participants_ozon/train_data.parquet')

test_pairs = pd.read_parquet('../hackathon_files_for_participants_ozon/test_pairs_wo_target.parquet')
test_data = pd.read_parquet('../hackathon_files_for_participants_ozon/test_data.parquet')

In [3]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(312,)


In [4]:
test_data['ebi_vector'] = test_data['name'].progress_apply(lambda x: embed_bert_cls(x, model, tokenizer))

  0%|          | 0/35730 [00:00<?, ?it/s]

In [6]:
test_data[['variantid', 'ebi_vector']].to_pickle('../hackathon_files_for_participants_ozon/test_data_rubert_tiny_2.pickle')

In [7]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')

def embed_bert_cls(text):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from more_itertools import chunked

test_embeddings = []

for chunk in tqdm(chunked(test_data['name'].values, n=16), total=len(test_data) // 16):
    test_embeddings.append(embed_bert_cls(chunk))

  0%|          | 0/2233 [00:00<?, ?it/s]

In [11]:
test_embeddings_all = np.concatenate(test_embeddings)

In [12]:
test_data['embedding'] = list(test_embeddings_all)

In [13]:
test_data[['variantid', 'embedding']].to_pickle(
    '../hackathon_files_for_participants_ozon/test_bert_embeddings_768.pickle'
)